In [1]:
import pyodbc 

In [5]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=mtama.oneacrefund.org,6543;'
                      'Database=RosterDataWarehouse;'
                      'Trusted_Connection=yes;')


InterfaceError: ('28000', '[28000] [Microsoft][ODBC SQL Server Driver][SQL Server]Login failed. The login is from an untrusted domain and cannot be used with Integrated authentication. (18452) (SQLDriverConnect); [28000] [Microsoft][ODBC SQL Server Driver][SQL Server]Login failed. The login is from an untrusted domain and cannot be used with Integrated authentication. (18452)')

In [ ]:
c = conn.cursor()
c.execute('SELECT * FROM database_name.table')

for row in cursor:
    print(row)